<a href="https://colab.research.google.com/github/dlawork9888/tensorflow_playground/blob/main/DynamicSequenceInputProcessTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ragged Tensor
- 비정형 텐서
- 다루기 꽤나 까다롭다

In [ ]:
import tensorflow as tf
import numpy as np

digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])
words = tf.ragged.constant([["So", "long"], ["thanks", "for", "all", "the", "fish"]])
print(tf.add(digits, 3))
print(tf.reduce_mean(digits, axis=1))
print(tf.concat([digits, [[5, 3]]], axis=0))
print(tf.tile(digits, [1, 2]))
print(tf.strings.substr(words, 0, 2))
print(tf.map_fn(tf.math.square, digits))


# tf.ragged.constant가 [길이가 다 다른 리스트들]을 비정형텐서로 바꾸는 메인인 것 같다.
# 하지만 느린 것 같았어 .

<tf.RaggedTensor [[6, 4, 7, 4], [], [8, 12, 5], [9], []]>
tf.Tensor([2.25              nan 5.33333333 6.                nan], shape=(5,), dtype=float64)
<tf.RaggedTensor [[3, 1, 4, 1], [], [5, 9, 2], [6], [], [5, 3]]>
<tf.RaggedTensor [[3, 1, 4, 1, 3, 1, 4, 1], [], [5, 9, 2, 5, 9, 2], [6, 6], []]>
<tf.RaggedTensor [[b'So', b'lo'], [b'th', b'fo', b'al', b'th', b'fi']]>
<tf.RaggedTensor [[9, 1, 16, 1], [], [25, 81, 4], [36], []]>


In [ ]:
# ragged.constant 실험

# 가변 크기의 이차원 비정형 텐서도 만들 수 있을까?
temp = []
for _ in range(1000):
  axis0 = np.random.randint(0,100)
  axis1 = np.random.randint(0,100)
  arr = np.random.randint(1,10,size = (axis0, axis1))
  temp.append(arr)

print(f'temp[0]:{temp[0]}')

temp_ragged_tensor = tf.ragged.constant(temp)

print(f"temp_ragged_tensor: {temp_ragged_tensor.shape}")

### 만들수 있다 !
### int (1000, 최대100, 최대100) 만드는데 10초 정도

temp[0]:[[1 4 6 ... 6 8 1]
 [7 5 9 ... 8 1 7]
 [2 4 8 ... 9 5 2]
 ...
 [2 8 9 ... 5 6 3]
 [9 4 6 ... 3 3 3]
 [9 2 9 ... 5 5 6]]
temp_ragged_tensor: (1000, None, None)


# Dynamic Sequence Input Process Tutorial
- 수열을 구별하는 신경망을 만들어보자
- 수열 => 시퀀스
- 수열은 두 종류, 하나는 diff가 홀수, 하나는 diff가 짝수

In [1]:
### 쉬운 예제부터

# 수열 예제 생성
# diff는 1 ~ 10
# 시작은 0 ~ 100
# 길이는 512 ~ 1024
# 각 수열은 1000개 씩

import tensorflow as tf
import numpy as np

### 넘파이는 비정형 배열(ex. shape = (100,None,None))을 다룰 수가 없다 !
#
sample_size = 1000

# 홀수 공차 등차수열 생성
odds_set = []
odds_row_lengths = [] #
for _ in range(sample_size):
  difference = np.random.randint(1,10) * 2 - 1
  start = np.random.randint(1, 100)
  seq_len = np.random.randint(512, 1024)
  seq = np.array([start + difference * x for x in range(seq_len)])
  odds_set.append(seq)
  odds_row_lengths.append(seq_len) #

# 짝수 공차 등차수열 생성
evens_set = []
evens_row_lengths = [] #
for _ in range(sample_size):
  difference = np.random.randint(1,10) * 2
  start = np.random.randint(1, 100)
  seq_len = np.random.randint(512, 1024)
  seq = np.array([start + difference * x for x in range(seq_len)])
  evens_set.append(seq)
  evens_row_lengths.append(seq_len) #

# 레이블 생성
odds_label = np.full((sample_size, ), 1,dtype=int)
evens_label = np.full((sample_size, ), 0,dtype=int)

# regged 로 변경
odds_set_ragged = tf.ragged.constant(odds_set)
evens_set_ragged = tf.ragged.constant(evens_set)

# concat
data = tf.concat([odds_set_ragged, evens_set_ragged], axis = 0)
labels = np.concatenate([odds_label, evens_label], axis = 0)
print(f'data.shape: {data.shape}')
print(f'data.shape[0]: {data.shape[0]}')


# Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices((data, labels))

# dataset 셔플
dataset = dataset.shuffle(buffer_size=len(dataset))

# dataset 확인
for datapoint in dataset.take(5):
  seq, label = datapoint
  print('-----------------------------------')
  print(f'seq len: {len(seq)}')
  print(f'label: {label}')
  print('-----------------------------------')

# 트 발 테 분리
# 데이터셋 크기 확인 및 나누기
dataset_size = tf.data.experimental.cardinality(dataset).numpy()  # 사이즈 가져오기 ! (shape속성이 존재하지 않음)
print(f'dataset_size: {dataset_size}')
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)

# 배치 처리
batch_size = 16
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

data.shape: (2000, None)
data.shape[0]: 2000
-----------------------------------
seq len: 850
label: 0
-----------------------------------
-----------------------------------
seq len: 806
label: 0
-----------------------------------
-----------------------------------
seq len: 693
label: 1
-----------------------------------
-----------------------------------
seq len: 723
label: 1
-----------------------------------
-----------------------------------
seq len: 514
label: 1
-----------------------------------
dataset_size: 2000


In [2]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import numpy as np

# RaggedTensor 입력을 처리하는 모델 생성
input_ragged = Input(shape=(None,), dtype=tf.int32, ragged=True)
embedding = Embedding(input_dim=20000, output_dim=64)(input_ragged)
lstm = LSTM(64)(embedding)
output = Dense(1, activation='sigmoid')(lstm)

model = Model(inputs=input_ragged, outputs=output)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 64)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1313089 (5.01 MB)
Trainable params: 1313089 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
100/100 [==============================] - 72s 613ms/step - loss: 0.6892 - accuracy: 0.5238
Epoch 2/10
100/100 [==============================] - 64s 638ms/step - loss: 0.5453 - accuracy: 0.7575
Epoch 3/10
100/100 [==============================] - 67s 660ms/step - loss: 0.2413 - accuracy: 0.9181
Epoch 4/10
100/100 [==============================] - 61s 615ms/step - loss: 0.0927 - accuracy: 0.9700
Epoch 5/10
100/100 [==============================] - 63s 630ms/step - loss: 0.0407 - accuracy: 0.9912
Epoch 6/10
100/100 [==============================] - 60s 598ms/step - loss: 0.0190 - accuracy: 0.9950
Epoch 7/10
100/100 [==============================] - 61s 614ms/step - loss: 0.0105 - accuracy: 0.9969
Epoch 8/10
100/100 [==============================] - 62s 623ms/step - loss: 0.0075 - accuracy: 0.9975
Epoch 9/10
100/100 [==============================] - 60s 605ms/step - loss: 0.0040 - accuracy: 0.9987
Epoch 10/10
100/100 [==============================] - 61s 613ms/step - l